# **3.3 Extraction of Indicative Adjective Phrases**

In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
import os
import matplotlib.pyplot as plt
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
from sklearn.feature_extraction.text import CountVectorizer
from yellowbrick.text import FreqDistVisualizer
from yellowbrick.datasets import load_hobbies
from nltk.corpus import stopwords
nltk.download('stopwords')
import seaborn as sns
import spacy
import collections


import string
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
import os
from nltk import FreqDist
import random

import en_core_web_sm
from nltk.util import ngrams

nlp = en_core_web_sm.load()
from operator import itemgetter

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.datasets.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.datasets. Anything that cannot be imported from sklearn.datasets is now part of the private API.
  warnings.warn(message, FutureWarning)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
reviews = pd.read_json('/content/reviewSelected100 2.json', encoding = "ISO-8859-1", lines = True)
reviews.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,8aoJJdKEO3ypoZNszpPu7Q,bGgAL09pxLnV_FFgR4ZADg,ZBE-H_aUlicix_9vUGQPIQ,5,0,0,0,We had my Mother's Birthday Party here on 10/2...,2016-11-09 20:07:25
1,J5NOCLdhuhor7USRhtYZ8w,pFCb-1j6oI3TDjr26h2cJQ,e-YnECeZNt8ngm0tu4X9mQ,4,0,0,0,Good Korean grill near Eaton Centre. The marin...,2015-12-05 05:06:43
2,PXiLWAYRt3xnHaJ8MB4rzw,mEzc6LeTNiQgIVsq3poMbg,j7HO1YeMQGYo3KibMXZ5vg,5,2,1,3,Was recommended to try this place by few peopl...,2014-10-11 05:16:15
3,VrLarvxZYJm74yAqtpe9PQ,o-zUN2WEZgjQS7jnNsec0g,7e3PZzUpG5FYOTGt3O3ePA,3,0,0,0,Ambience: Would not expect something this nice...,2016-07-25 03:45:26
4,C1CUpidlVFprUCkApqzCmA,Wlx0iBXJvk4x0EeOt2Bz1Q,vuHzLZ7nAeT-EiecOkS5Og,1,11,0,3,Absolutely the WORST pool company that I have ...,2016-04-11 18:49:11


# Generating a random business ID

In [3]:
rand_bid = reviews['business_id'].sample()
rand_bid_str = rand_bid.to_string(index = False).strip()
b_id_grouped = reviews.groupby(['business_id'])
rand_biz = b_id_grouped.get_group(rand_bid_str)
rand_biz = rand_biz.reset_index()

In [4]:
rand_biz['tokenize'] = rand_biz['text'].apply(nltk.word_tokenize)
rand_biz['pos_tag'] = rand_biz['tokenize'].apply(nltk.pos_tag)
# rand_biz.to_json(r'/rand_biz_review.json', orient='records', lines=True)
# tokenized_path = '/rand_biz_review.json'
# rand_adj_df = pd.read_json(tokenized_path, lines=True,encoding = "ISO-8859-1")

In [5]:
def reviews_extract(biz_id):
  extracted_reviews = []
  for idx, biz in reviews.iterrows():
    if biz['business_id'] == biz_id:
        extracted_reviews.append(biz['text'])
  return extracted_reviews 
reviews_txt = reviews_extract(rand_bid_str)

# Tokenization and POS Tagging

In [6]:
def tok_pos(reviews_txt: list):
    processed_txt = []
    x = nlp(str(reviews_txt))
    for token in x:
        process = []
        process.append(token.text)
        process.append(token.pos_)
        #print(token.pos_)
        processed_txt.append(process)
    return processed_txt,x
processed_txt,result = tok_pos(reviews_txt)

# Adjective Phrase Extraction

In [7]:
def adj_phrases_bigram(token_list):
    adj_bigram = adj_trigram = adj_ngram = []
    count_bi = count_tri = count_n = 0
    for idx in range(len(token_list)-2):
        temp_bi = []
        temp_n = []
        temp_tri = []
        prev = token_list[idx-1]
        current = token_list[idx]
        next = token_list[idx+1]
        next_next = token_list[idx+2]
        if(((prev.pos_) == 'ADV'and (current.pos_ == 'ADJ')) or ((prev.pos_) == 'ADJ'and (current.pos_ == 'ADV')) or ((prev.pos_) == 'ADJ'and (current.pos_ == 'ADJ'))):
            count_bi += 1
            temp_bi.append(prev.text)
            temp_bi.append(current.text)
        temp_bi = tuple(temp_bi)
        adj_bigram.append(temp_bi)
        adj_bigram = list(filter(lambda x: x, adj_bigram))
        if(((prev.pos_) == 'ADJ' and (current.pos_ == 'ADP') and (next.pos_ == 'NOUN')) or ((prev.pos_) == 'ADV'and (current.pos_ == 'ADV') and (next.pos_ == 'ADJ'))):
            count_tri += 1
            temp_tri.append(prev.text)
            temp_tri.append(current.text)
            temp_tri.append(next.text)
        temp_tri = tuple(temp_tri)
        adj_trigram.append(temp_tri)
        adj_trigram = list(filter(lambda x: x, adj_trigram))
        if((prev.pos_) == 'ADJ' and (current.pos_ == 'ADP') and (next.pos_ == 'ADP') and next_next.pos_ == 'NOUN'):
            count_n += 1
            temp_n.append(prev.text)
            temp_n.append(current.text)
            temp_n.append(next.text)
            temp_n.append(next_next.text)
        temp_n = tuple(temp_n)
        adj_ngram.append(temp_n)
        adj_ngram = list(filter(lambda x: x, adj_ngram))
    return adj_bigram, count_bi, adj_trigram, count_tri, adj_ngram, count_n

In [8]:
adj_bigram, count_bi, adj_trigram, count_tri, adj_ngram, count_n = adj_phrases_bigram(result)
print("Bigram adjective phrases")
print(count_bi)
print(adj_bigram)
print("Trigram adjective phrases")
print(count_tri)
print(adj_trigram)
print("N-gram adjective phrases")
print(count_n)
print(adj_ngram)

Bigram adjective phrases
279
[('very', 'contemporary'), ('very', 'nice'), ('incredibly', 'sensitive'), ('so', 'careful'), ('different', 'oral'), ('too', 'nice'), ('how', 'friendly'), ('comfortable.\\n\\nThe', 'next'), ('next', 'big'), ('so', 'hard'), ('super', 'interesting'), ('more', 'dental'), ('incredibly', 'clean'), ('standard', 'dental'), ('so', 'glad'), ('so', 'amazing'), ('very', 'helpful'), ('very', 'professional'), ('other', 'dentist'), ('good', 'too'), ('Best', 'dental'), ('really', 'knowledgeable'), ('knowledgeable', 'caring'), ('best', 'out'), ('extremely', 'knowledgeable'), ('too', 'cute'), ('so', 'nice'), ('very', 'informative'), ('VERY', 'nice'), ('so', 'cool'), ('other', 'dental'), ('pretty', 'awesome'), ('SO', 'HAPPY'), ('very', 'amiable'), ('sold.\\n\\nThe', 'entire'), ('so', 'friendly'), ('super', 'gentle'), ('most', 'modern'), ('past', 'few'), ('pleasantly', 'surprised'), ('how', 'beautiful'), ('extremely', 'clean'), ('super', 'friendly'), ('more', 'comfortable'), (

# Indicativeness

In [9]:
def adj_ind (adj_phrase: list):
    freq_dist = nltk.FreqDist(adj_phrase)
    freq_dist_top20 = freq_dist.most_common(20)
    freq_dict = {}
    for i in freq_dist.most_common(20):
      freq = i[1]/len(reviews_txt)
      freq_dict[i[0]] = freq
    return freq_dict

In [10]:
freq_bi = adj_ind(adj_bigram)
freq_tri = adj_ind(adj_trigram)
frequency = {**freq_bi, **freq_tri}
frequency_top_10 = dict(sorted(frequency.items(), key = itemgetter(1), reverse = True)[:10])
frequency_top_10

{('extremely', 'clean'): 0.02,
 ('most', 'modern'): 0.02,
 ('so', 'gentle'): 0.04,
 ('so', 'glad'): 0.05,
 ('so', 'nice'): 0.06,
 ('so', 'sweet'): 0.03,
 ('very', 'gentle'): 0.03,
 ('very', 'nice'): 0.07,
 ('very', 'professional'): 0.02,
 ('very', 'thorough'): 0.03}

# Across all businesses

In [11]:
biz_list = []
for i in range(len(reviews)):
    biz = reviews.iloc[i]
    biz_list.append(biz['business_id'])
biz_list = set(biz_list)

In [12]:
biz_reviews = {}
for biz in biz_list:
    #extract reviews for each individual id
    rev_txt = reviews_extract(biz)
    biz_reviews[biz] = rev_txt

In [13]:
for i in biz_reviews:
    processed_biz, result_biz = tok_pos(biz_reviews[i])
    # print(result_biz)
    adj_bigram, count_bi, adj_trigram, count_tri, adj_ngram, count_n = adj_phrases_bigram(result_biz)
    # print(adj_bigram)
    fin_freq_bi = adj_ind(adj_bigram)
    fin_freq_tri = adj_ind(adj_trigram)
    fin_frequency = {**fin_freq_bi, **fin_freq_tri}
    # print(fin_frequency)
    biz_reviews[i] = fin_frequency

In [14]:
for i in biz_reviews:
    print(i)
    print(biz_reviews[i])

CQxhyLM833WF45yrS9yeAA
{('very', 'grateful'): 0.06, ('so', 'grateful'): 0.06, ('so', 'kind'): 0.06, ('very', 'difficult'): 0.05, ('as', 'much'): 0.05, ('most', 'comfortable'): 0.05, ('so', 'thankful'): 0.05, ('so', 'much'): 0.04, ('so', 'glad'): 0.04, ('more', 'comfortable'): 0.04, ('most', 'difficult'): 0.03, ('more', 'peaceful'): 0.03, ('so', 'peaceful'): 0.03, ('very', 'compassionate'): 0.03, ('very', 'sad'): 0.03, ('very', 'peaceful'): 0.03, ('extremely', 'compassionate'): 0.03, ('very', 'gentle'): 0.03, ('very', 'old'): 0.03, ('enough', 'good'): 0.02, ('comfortable', 'at', 'home'): 0.03, ('so', 'very', 'grateful'): 0.02, ('full', 'of', 'empathy'): 0.01, ('much', 'more', 'unbearable'): 0.01, ('calm', 'at', 'home'): 0.01, ('sick', 'for', 'months'): 0.01, ('just', 'as', 'much'): 0.01, ('steady', 'on', 'prescription'): 0.01, ('always', 'so', 'terrified'): 0.01, ('only', 'very', 'informative'): 0.01, ('still', 'very', 'close'): 0.01, ('also', 'so', 'kind'): 0.01, ('obviously', 'very', 

# Comparison

In [15]:
frequency_top_10_phrases = list(frequency_top_10)
frequency_top_10_indicativeness = list(frequency_top_10.values())

In [16]:
for idx in range(10):
  print("For the business: ", rand_bid_str)
  print("For the phrase: ", frequency_top_10_phrases[idx])
  print("\n Other businesses")
  for i in biz_reviews:
      freq_list = biz_reviews[i]
      for freq in freq_list:
          if(freq == frequency_top_10_phrases[idx]):
              print("Business: "+ i + " Probability: ", biz_reviews[i][freq])
              if(float(frequency_top_10_indicativeness[idx]) - biz_reviews[i][freq]) >= 0.03:
                  print("Significant difference in indicativeness of the given adjective phrase\n")
  print("\n_______________________________________________________\n")

For the business:  YgWGWza3sXzI0brcOogR5A
For the phrase:  ('very', 'nice')

 Other businesses
Business: pJnN5PxqFPGZW2pJWkQtzA Probability:  0.02
Significant difference in indicativeness of the given adjective phrase

Business: zZ7KDK3GAkBUZzsaqB1A4Q Probability:  0.01
Significant difference in indicativeness of the given adjective phrase

Business: iK_3WktnHa_YlbWdA0Axkw Probability:  0.03
Significant difference in indicativeness of the given adjective phrase

Business: bmm5F5smQrHCEkFYjwrQ9w Probability:  0.03
Significant difference in indicativeness of the given adjective phrase

Business: mZEj6ggkh2l2QlbCJACaYw Probability:  0.07
Business: a4GRh1TlOVhPD401mSPLZg Probability:  0.06
Business: KjjNv4YFzBFnsBh2ncH1Iw Probability:  0.03
Significant difference in indicativeness of the given adjective phrase

Business: yAkaDc6MWX3dy8bnEcCrRQ Probability:  0.04
Significant difference in indicativeness of the given adjective phrase

Business: dcrL22ant20s1thjdVGhLw Probability:  0.03
Signi